### **Nome:** Luís Coimbra
### **Cadeira:** Otimização em redes e redes sociais
### **Dataset:** The Marvel Universe Social Network


O dataset selecionado para este trabalho é "The Marvel Universe Social Network", que consiste em três arquivos CSV. O **objetivo deste estudo** é analisar as relações existentes entre os personagens da Marvel, destacando quais os heróis que possuem um número maior de conexões.

A análise destas relações pode revelar informações interessantes sobre a estrutura social da Marvel, como **grupos de personagens que frequentemente interagem entre si**, **personagens centrais que se conectam com diversos heróis** e até mesmo a **existência de comunidades ou subgrupos dentro do universo Marvel**.

Além disso, devido à atualização do universo Marvel por meio de um dos filmes mais recentes, certos personagens serão removidos do dataset original. Isso permitirá que examinemos como essas alterações afetam as relações existentes e identifiquemos novos vínculos que possam surgir.

Espera-se que esta análise proporcione uma compreensão mais aprofundada das conexões entre os personagens da Marvel, permitindo-nos visualizar a complexidade das interações sociais dentro desse universo.

**nodes.csv** = 2 colunas ( node, type) , indica o nome e o tipo(pode ser herói ou comic) de node

**edges.csv** = 2 colunas ( hero,comic) e indica em que comics os heróis aparecem

**hero-edge.csv** = 2 colunas com a conexão entre os heróis

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from itertools import combinations
import plotly.graph_objects as go

# **1 -  Análise exploratórios dos dados**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
edge_df = pd.read_csv("/content/drive/MyDrive/Mestrado/4 trimestre/ORRS/Trabalho/edges.csv")
hero_net_df=pd.read_csv("/content/drive/MyDrive/Mestrado/4 trimestre/ORRS/Trabalho/hero-network.csv")
nodes = pd.read_csv("/content/drive/MyDrive/Mestrado/4 trimestre/ORRS/Trabalho/nodes.csv")

Nos datasets seguintes podemos verificar que nas colunas de "hero" estão representados os **nomes das personagens de heróis / nome real da pessoa fictícia**.

In [ ]:
edge_df.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [ ]:
hero_net_df.head(5)

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


É **importante** ressaltar que , ao analisar o dataset , que se baseia sobre as comics , é comum encotnrar diferentes representações de certas personagens , como o IRON MAN , que é retratado por personagens fictícias diferentes como Tony Stark ( Original ) e James R.

In [ ]:
nodes.head()

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


### **A)**  Quantos Spider-Man existem ?

Dado que o Spider-Man consiste no meu herói favorito , reparei que existem diferentes tipos do mesmo, sendo que apliquei as seguintes linhas de código para perceber no total quantos existem (neste caso 12 )

In [ ]:
spider = pd.DataFrame({'Diferentes tipos de Spider-Man':sorted([h for h in edge_df['hero'].unique() if 'SPIDER' in h])}),
                     # "h for h in" cria uma nova variável com certas condições
                     # sendo que essa condição é o facto de na coluna hero ter a palavra SPIDER
                     # esta por ordem tmb (sorted )

display(spider)

(   Diferentes tipos de Spider-Man
 0                   BEACH, SPIDER
 1                   BLOOD SPIDER/
 2            MAN-SPIDER CLONE | M
 3             MAN-SPIDER | MUTANT
 4            SPIDER-MAN CLONE/BEN
 5            SPIDER-MAN III/MARTH
 6         SPIDER-MAN/PETER PARKER
 7            SPIDER-WOMAN DOPPELG
 8            SPIDER-WOMAN II/JULI
 9            SPIDER-WOMAN IV/CHAR
 10           SPIDER-WOMAN/JESSICA
 11                     SPIDERCIDE
 12           STEEL SPIDER/OLLIE O,)

### **B)**  Quantos Iron-Man existem ?

In [ ]:
iron = pd.DataFrame({'Diferentes tipos de Iron-Man':sorted([i for i in edge_df['hero'].unique() if 'IRON MAN' in i])}),
                     # "i for i in" cria uma nova variável com certas condições
                     # sendo que essa condição é o facto de na coluna hero ter a palavra IRON MAN
                     # esta por ordem tmb( sorted )

display(iron)

(  Diferentes tipos de Iron-Man
 0               IRON MAN ARMOR
 1         IRON MAN DOPPELGANGE
 2         IRON MAN III/EDDIE M
 3         IRON MAN IV/JAMES R.
 4         IRON MAN V/TEEN TONY
 5          IRON MAN/TONY STARK
 6         KREE IRON MAN [KREE],)

### **C)** Quantos Hulks existem?

In [ ]:
hulk = pd.DataFrame({'Diferentes tipos de Hulk':sorted([p for p in edge_df['hero'].unique() if 'HULK' in p])}),
                     # "p for p in" cria uma nova variável com certas condições
                     # sendo que essa condição é o facto de na coluna hero ter a palavra HULK
                     # esta por ordem tmb ( sorted )

display(hulk)

(   Diferentes tipos de Hulk
 0        BEDFORD, BRAD HULK
 1         HULK DOPPELGANGER
 2      HULK DOPPELGANGER II
 3      HULK III/BRUCE BANNE
 4                   HULK IV
 5                HULK ROBOT
 6      HULK | MUTANT X-VERS
 7      HULK/DR. ROBERT BRUC
 8      SHE-HULK DOPPELGANGE
 9      SHE-HULK/JENNIFER WA
 10                    SHULK,)

### **D)** Conexões

O número de conexões existentes pode variar de acordo com o facto de o herói aparecer na coluna hero1 ou hero2. De seguida são dados alguns exemplos de modo a demonstrar estes casos.

In [ ]:
# De modo a realizar esta experiência é importante perceber que devem ser utilizados apenas os nomes que se apresentam antes do "/", logo é aplicado a seguinte linha de código.

for c in ['hero1', 'hero2']:
    hero_net_df[c] = hero_net_df[c].apply(lambda x: x.split("/")[0])
edge_df['hero'] = edge_df['hero'].apply(lambda x: x.split("/")[0])

### **Spider-Man** e **Iron-Man**

In [ ]:
print("Quantidade de conexões entre Spider-Man e Iron-Man")
print()

# De seguida são criadas linhas de código para analisar o número de vezes que o spider man ou Hulk aparecem na coluna hero1
xspider1=len(hero_net_df[(hero_net_df['hero1']=='SPIDER-MAN')&(hero_net_df['hero2']=='IRON MAN')])
xironman1=len(hero_net_df[(hero_net_df['hero1']=='IRON MAN')&(hero_net_df['hero2']=='SPIDER-MAN')])

print("hero1 = SPIDER-MAN | hero2 = IRON-MAN :", xspider1)
print("hero1 = IRON-MAN | hero2 = SPIDER-MAN :", xironman1)

print("Total Connections:",xspider1 + xironman1)

Quantidade de conexões entre Spider-Man e Iron-Man

hero1 = SPIDER-MAN | hero2 = IRON-MAN : 40
hero1 = IRON-MAN | hero2 = SPIDER-MAN : 54
Total Connections: 94


### **Spider-Man** e **Hulk**

In [ ]:
print("Quantidade de conexões entre Spider-Man e Hulk")
print()

# De seguida são criadas linhas de código para analisar o número de vezes que o spider man ou Hulk aparecem na coluna hero1
spider1=len(hero_net_df[(hero_net_df['hero1']=='SPIDER-MAN')&(hero_net_df['hero2']=='HULK')])
hulk1=len(hero_net_df[(hero_net_df['hero1']=='HULK')&(hero_net_df['hero2']=='SPIDER-MAN')])

print("hero1 = SPIDER-MAN | hero2 = HULK :", spider1)
print("hero1 = HULK | hero2 = SPIDER-MAN :", hulk1)

print("Total connections:",spider1+hulk1)

Quantidade de conexões entre Spider-Man e Hulk

hero1 = SPIDER-MAN | hero2 = HULK : 43
hero1 = HULK | hero2 = SPIDER-MAN : 50
Total connections: 93


### **Hulk** e **Iron-Man**

In [ ]:
print("Quantidade de conexões entre Spider-Man e Hulk")
print()

# De seguida são criadas linhas de código para analisar o número de vezes que o spider man ou Hulk aparecem na coluna hero1
zhulk1=len(hero_net_df[(hero_net_df['hero1']=='HULK')&(hero_net_df['hero2']=='IRON MAN')])
zironman1=len(hero_net_df[(hero_net_df['hero1']=='IRON MAN')&(hero_net_df['hero2']=='HULK')])

print("hero1 = IRON-MAN | hero2 = HULK :", zhulk1)
print("hero1 = HULK | hero2 = IRON-MAN :", zironman1)

print("Total connections:",zhulk1 + zironman1)

Quantidade de conexões entre Spider-Man e Hulk

hero1 = IRON-MAN | hero2 = HULK : 52
hero1 = HULK | hero2 = IRON-MAN : 33
Total connections: 85


### **E)** Top 30 heróis com mais comics

In [ ]:
top30_hero = edge_df.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=False).head(30)
display(top30_hero)

,comic
hero,
SPIDER-MAN,1577
CAPTAIN AMERICA,1334
IRON MAN,1150
THING,963
THOR,956
HUMAN TORCH,886
MR. FANTASTIC,854
HULK,835
WOLVERINE,819


###  **F)**Top 5 heróis com menos comics

In [ ]:
less5_hero = edge_df.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=True).head(5)
display(less5_hero)

,comic
hero,
24-HOUR MAN,1
"VERSCHLAGEN, FRAU",1
"LUMLEY, JUDY",1
"DEVEREAUX, IRIS",1
"STROUGHTON, WILLIAM",1


# **2 -  Contrução da rede( Antes do Snap do Thanos)**


Nas comics é apresentado um vilão que tem como propósito eliminar 50% da população. Dado a concretização do seu objetivo decidi construir duas redes:

1- A primeira será a analise dos relacionamentos entre heróis antes do Thanos eliminar 50 % da população

2- Depois será criada outra rede mas eliminando as personagens que morreram nas comics( informação retirada da internet)

### **Top 10 heróis com mais conexões**

A seguinte informação é a totalidade de conexões que se apresentam pelos heróis e podemos ver que o Capitão América está situado em primeiro lugar.

In [ ]:
# Número de conexões para cada herói
connections = hero_net_df['hero1'].value_counts() + hero_net_df['hero2'].value_counts()

# top 25 heróis com mais conexões
top_10_heroes = connections.nlargest(10)


# Resultados
print("Top 10 heróis com maix conexões:")
print("")
print(top_10_heroes.reset_index().rename(columns={'index': 'Hero', 0: 'Connections'}))

Top 10 heróis com maix conexões:

              Hero  Connections
0  CAPTAIN AMERICA      16499.0
1       SPIDER-MAN      13717.0
2         IRON MAN      11817.0
3             THOR      11427.0
4            THING      10681.0
5        WOLVERINE      10353.0
6      HUMAN TORCH      10237.0
7    SCARLET WITCH       9911.0
8    MR. FANTASTIC       9775.0
9          VISION        9696.0


### **Fazer peso das arestas(conexões entre heróis)**


In [ ]:
topn = 30 # fazer top 30 porque os heróis todos ia ser muito pesado
topn_hero = edge_df.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=False).head(topn).index
# agrupar a informação no dataset "edge_df" por hero e conta-se as vezes que os heróis aparecem na comic . Poem-se por ordem decrescente.

hero1_ = []; hero2_ = []; cont_ = []; # criação de lists vazias para agrupar combinações de herois e as suas counts
for comb in list(combinations(topn_hero, 2)):    # combinações de heróis com lenght de 2
    temp1 = set(edge_df[edge_df['hero']==comb[0]]['comic']) # criação de apariçoes para o primeiro herói
    temp2 = set(edge_df[edge_df['hero']==comb[1]]['comic']) # criação de aparições para o segundo herói
    cnt = len(temp1.intersection(temp2)) # # Calcula o número de comics em que os heróis apareçem juntos. Interseção de aparições do herói 1 com herói 2.
    hero1_.append(comb[0]); hero2_.append(comb[1]); cont_.append(cnt); # adicionar às colunas a sua respetiva informação

herois_df = pd.DataFrame({'Hero1':hero1_, 'Hero2':hero2_, 'CONT':cont_})


sorted_df = herois_df.sort_values(by='CONT', ascending=False)
display(sorted_df.head(20))


,Hero1,Hero2,CONT
85,THING,HUMAN TORCH,724
135,HUMAN TORCH,MR. FANTASTIC,694
86,THING,MR. FANTASTIC,690
161,MR. FANTASTIC,INVISIBLE WOMAN,682
138,HUMAN TORCH,INVISIBLE WOMAN,675
89,THING,INVISIBLE WOMAN,650
12,SPIDER-MAN,"WATSON-PARKER, MARY",614
18,SPIDER-MAN,"JAMESON, J. JONAH",514
29,CAPTAIN AMERICA,IRON MAN,440
250,SCARLET WITCH,VISION,426


Os heróis **Thing** e **Human Torch** são os que apresentam mais conexões um com o outro.

## **Criação do Grafo**

### **-Cores**

In [ ]:
# Atribui cores aos top 10 heróis com mais conexões

HERO_COLOR = {'CAPTAIN AMERICA':'darkblue','IRON MAN':'yellow','SPIDER-MAN':'red','THOR':'violet','THING': 'orange','WOLVERINE':'darkgrey','HUMAN TORCH':'purple',
              'INVISIBLE WOMAN':'black','MR. FANTASTIC': 'pink','VISION':'blue'}

# - https://towardsdatascience.com/tutorial-network-visualization-basics-with-networkx-and-plotly-and-a-little-nlp-57c9bbb55bb9

marvel_net = nx.Graph() #criação de um grafo não direcionado
for i, row in herois_df.iterrows(): # atribui o dice da linha à variável "i" e os valores da linha às variáveis "row"
    marvel_net.add_edge(row['Hero1'], row['Hero2'], weight=row['CONT'])  # adicionar aresta com os valores corretos para nodos e arestas

# Criação do layour "Spring" do NetworkX
layout_marvel = nx.spring_layout(marvel_net,seed=5)

In [ ]:
# Função para criar uma aresta entre os nodes
def make_edge(x, y, text, width):
    return  go.Scatter(x=x, y=y, line=dict(width=width, color='lightgray'), hoverinfo='text', text=([text]), mode='lines')

# lista edge_trace para armazendar todas as arestas do gráfico
edge_trace = []
for edge in marvel_net.edges():
    if marvel_net.edges()[edge]['weight'] > 0: # evita arestas com peso 0 no gráfico
        char_1 = edge[0] #nós conectados
        char_2 = edge[1] #nós conectados
        x0, y0 = layout_marvel[char_1] # variáveis obtidas a partir das cordenadas dos nós conectados
        x1, y1 = layout_marvel[char_2]
        trace  = make_edge([x0, x1, None], [y0, y1, None], None, width=5*(marvel_net.edges()[edge]['weight']/herois_df['CONT'].max())) # largura calculado normalizando o peso pelo valor máximo da coluna de contagem
        edge_trace.append(trace) # chama-mos a função make_edge2 passando-lhe as coordenadas dos nós(x0...) , o texto como None

### -  **Between Centrality**

### -  **Closeness Centrality**

### -  **Eigenvector Centrality**

### -  **Page Rank**

In [ ]:
cent_df = pd.DataFrame(index=list(marvel_net.nodes()))

In [ ]:
#Between Centrality
between_ = nx.betweenness_centrality(marvel_net, weight='weight')
cent_df['Betweenness Centrality'] = pd.Series(index=[node for node, betweenness_centrality in between_.items()],data=[float(betweenness_centrality) for node, betweenness_centrality in between_.items()])

#Closeness Centrality
closeness_ = nx.closeness_centrality(marvel_net,distance="weight")   # closeness function tem um parametro distance
cent_df['Closeness Centrality'] = pd.Series(index=[node for node, closeness_centrality in closeness_.items()],data=[float(closeness_centrality) for node, closeness_centrality in closeness_.items()])

#Eigenvector Centrality
eigen_ = nx.eigenvector_centrality(marvel_net, weight='weight')
cent_df['Eigenvector Centrality'] = pd.Series(index=[node for node, eigenvector_centrality in eigen_.items()],data=[float(eigenvector_centrality) for node, eigenvector_centrality in eigen_.items()])

#PageRank

page_ = nx.pagerank(marvel_net, weight='weight')
cent_df['PageRank'] = pd.Series(index=[node for node, eigen_centrality in page_.items()],data=[float(page_centrality) for node, page_centrality in page_.items()])

display(cent_df)


,Betweenness Centrality,Closeness Centrality,Eigenvector Centrality,PageRank
SPIDER-MAN,0.000000,0.030526,0.140174,0.040601
CAPTAIN AMERICA,0.000000,0.041193,0.292802,0.053595
IRON MAN,0.000000,0.060417,0.249492,0.043453
THING,0.000000,0.048986,0.297003,0.046662
THOR,0.000000,0.068396,0.214586,0.036956
HUMAN TORCH,0.000000,0.042899,0.294740,0.046054
MR. FANTASTIC,0.000000,0.042899,0.290314,0.044625
HULK,0.000000,0.078804,0.109661,0.024427
WOLVERINE,0.000000,0.073232,0.142526,0.036160
INVISIBLE WOMAN,0.000000,0.068396,0.278668,0.042289


In [ ]:
# node trace para armazenar os nodes do gráfico
node_trace = go.Scatter(x=[], y=[], text=[], textposition="top center", textfont_size=10, mode='markers+text', hoverinfo='none',
                        marker=dict(color=[], size=[], line_width=[], line_color=[]))

# Para cada node fazer a posição e tamanho e adicionar a node_trace
for node in marvel_net.nodes():
    x, y = layout_marvel[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    color = 'gray'
    line_width = 2
    line_color = 'darkgray'
    name_text = node


# agora para personalizar os top 10 herois com cores

    if node in HERO_COLOR:
        color = HERO_COLOR[node]; line_color='black';

    if node in [v[0] for v in page_]:
        name_text = '<b>' + node + '</b>'

    node_trace['marker']['color'] += tuple([color])
    node_trace['marker']['size'] += tuple([int(400*page_[node])]) # node size is proportional to page rank
    node_trace['marker']['line_color'] += tuple([line_color])
    node_trace['text'] += tuple([name_text])



# Layout costumizável

layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # background transparente
    plot_bgcolor='rgba(0,0,0,0)', # 2 background transparente
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)

# Criar figura
fig = go.Figure(layout = layout)

# Adiciona todas as informação na lista de edge_trace
for trace in edge_trace:
    fig.add_trace(trace)

# Adiciona-se node_trace  ( tem informação dos nós)
fig.add_trace(node_trace)
# Remove-se as etiquetas
fig.update_layout(showlegend = False)
# Remove-se as etiquetas de x e y
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)


fig.update_layout(title=" <b>Network dos Top 30 heróis(antes do SNAP)<b>") # Adiciona-se título e os <b> servem para por a negrito
fig.show()


# **3 -  Contrução da rede( Depois do Snap do Thanos)**

Nesta parte do trabalho irei recriar o grafo anterior mas eliminando as personagens que morreram nas comics com o Thanos.
É importante realçar que a lista de mortes varia dependendo do universo em que estamos situados, pois o universo Marvel é vasto e complexo com diferentes continuidades e eventos alternativos.

No entanto , de modo a realizar o trabalho **irei me basear em uma das comics(universos) mais conhecidas**

In [ ]:
edge_df2 = pd.read_csv("/content/drive/MyDrive/Mestrado/4 trimestre/ORRS/Trabalho/edges.csv")
hero_net_df2=pd.read_csv("/content/drive/MyDrive/Mestrado/4 trimestre/ORRS/Trabalho/hero-network.csv")
nodes2 = pd.read_csv("/content/drive/MyDrive/Mestrado/4 trimestre/ORRS/Trabalho/nodes.csv")

In [ ]:
for c in ['hero1', 'hero2']:
    hero_net_df2[c] = hero_net_df2[c].apply(lambda x: x.split("/")[0])
edge_df2['hero'] = edge_df2['hero'].apply(lambda x: x.split("/")[0])

### **Heróis Removidos**

In [ ]:
herois_removidos = ['SPIDER-MAN', 'IRON MAN','THOR','CAPTAIN AMERICA','HULK','BLACK WIDOW','DR. STRANGE','SCARLET WITCH','VISION','GAMORA','GROOT']

nodes2 = nodes2[~nodes2['node'].isin(herois_removidos)] #usar a função isin() juntamente com o operador ~ para negar o resultado.
edge_df2 = edge_df2[~edge_df2['hero'].isin(herois_removidos)]
hero_net_df2 = hero_net_df2[~hero_net_df2[['hero1', 'hero2']].isin(herois_removidos).any(axis=1)]

#Ao usar any(axis=1), estamos a verificar se pelo menos um valor em qualquer uma das colunas satisfaz a condição para cada linha do DataFrame.
# Se algum valor atender à condição, a linha será considerada para exclusão.

Teste para verificar se um dos heróis removidos foi de facto removido

In [ ]:
spiderman_removed = ('SPIDER-MAN' not in hero_net_df2['hero1']) and ('SPIDER-MAN' not in hero_net_df2['hero2'])
spiderman_removed = spiderman_removed and ('SPIDER-MAN' not in edge_df2['hero'])
spiderman_removed = spiderman_removed and ('SPIDER-MAN' not in nodes2['node'])

print("O herói 'SPIDER-MAN' foi removido:", spiderman_removed)


O herói 'SPIDER-MAN' foi removido: True


### **A)**Top 30 heróis com mais comics(depois do snap)

In [ ]:
top30_hero_thanos = edge_df2.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=False).head(30)
display(top30_hero_thanos)

,comic
hero,
THING,963
HUMAN TORCH,886
MR. FANTASTIC,854
WOLVERINE,819
INVISIBLE WOMAN,762
BEAST,635
"WATSON-PARKER, MARY",622
DAREDEVIL,619
HAWK,605


###  **B)**Top 5 heróis com menos comics( continuam os mesmos)

In [ ]:
less5_hero_thanos = edge_df2.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=True).head(5)
display(less5_hero_thanos)

,comic
hero,
24-HOUR MAN,1
"GARFIELD, MURRAY",1
GARGOYLE,1
GARGOYLE | MUTANT X-,1
GARM,1


### **C)**Top 10 heróis com mais conexões

In [ ]:
# Número de conexões para cada herói
connections2 = hero_net_df2['hero1'].value_counts() + hero_net_df2['hero2'].value_counts()

# top 25 heróis com mais conexões
top_10_heroes2 = connections2.nlargest(10)


# Resultados
print("Top 10 heróis com maix conexões:")
print("")
print(top_10_heroes2.reset_index().rename(columns={'index': 'Hero', 0: 'Connections'}))

Top 10 heróis com maix conexões:

              Hero  Connections
0        WOLVERINE       9971.0
1            THING       9825.0
2      HUMAN TORCH       9421.0
3    MR. FANTASTIC       8972.0
4          CYCLOPS       8765.0
5  INVISIBLE WOMAN       8601.0
6            BEAST       8550.0
7            STORM       8516.0
8          VISION        7964.0
9      COLOSSUS II       7611.0


### **D)**Fazer peso das arestas(conexões entre heróis)

In [ ]:
topn2 = 30 # fazer top 30 porque os heróis todos ia ser muito pesado
topn_hero2 = edge_df2.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=False).head(topn2).index
# agrupar a informação no dataset "edge_df2" por hero e conta-se as vezes que os heróis aparecem na comic . Poem-se por ordem decrescente.

hero1_ = []; hero2_ = []; cont_ = []; # criação de lists vazias para agrupar combinações de herois e as suas counts
for comb in list(combinations(topn_hero2, 2)):    # combinações de heróis com lenght de 2
    temp1 = set(edge_df2[edge_df2['hero']==comb[0]]['comic']) # criação de apariçoes para o primeiro herói
    temp2 = set(edge_df2[edge_df2['hero']==comb[1]]['comic']) # criação de aparições para o segundo herói
    cnt = len(temp1.intersection(temp2)) # # Calcula o número de comics em que os heróis apareçem juntos. Interseção de aparições do herói 1 com herói 2.
    hero1_.append(comb[0]); hero2_.append(comb[1]); cont_.append(cnt); # adicionar às colunas a sua respetiva informação

herois_df2 = pd.DataFrame({'Hero1':hero1_, 'Hero2':hero2_, 'CONT':cont_})


sorted_df_2 = herois_df2.sort_values(by='CONT', ascending=False)
display(sorted_df_2.head(20))

,Hero1,Hero2,CONT
0,THING,HUMAN TORCH,724
29,HUMAN TORCH,MR. FANTASTIC,694
1,THING,MR. FANTASTIC,690
58,MR. FANTASTIC,INVISIBLE WOMAN,682
31,HUMAN TORCH,INVISIBLE WOMAN,675
3,THING,INVISIBLE WOMAN,650
246,WASP,ANT-MAN,399
94,WOLVERINE,STORM,389
232,CYCLOPS,MARVEL GIRL,384
318,STORM,COLOSSUS II,344


## **Criação do Grafo**

### **-Cores**

In [ ]:
# Atribui cores aos top 10 heróis com mais conexões


HERO_COLOR2 = {'THING':'darkblue','HUMAN TORCH':'yellow','MR. FANTASTIC':'red','WOLVERINE':'violet','INVISIBLE WOMAN': 'orange','BEAST':'darkgrey','STORM':'purple',
              'VISION':'black','COLOSSUS II': 'pink','CYCLOPS':'blue'}

# - https://towardsdatascience.com/tutorial-network-visualization-basics-with-networkx-and-plotly-and-a-little-nlp-57c9bbb55bb9

marvel_net2 = nx.Graph() #criação de um grafo não direcionado
for i, row in herois_df2.iterrows(): # atribui o dice da linha à variável "i" e os valores da linha às variáveis "row"
    marvel_net2.add_edge(row['Hero1'], row['Hero2'], weight=row['CONT'])  # adicionar aresta com os valores corretos para nodos e arestas

# Criação do layour "Spring" do NetworkX
layout_marvel2 = nx.spring_layout(marvel_net2,seed=11)

In [ ]:
# Função para criar uma aresta entre os nodes
def make_edge2(x, y, text, width):
    return  go.Scatter(x=x, y=y, line=dict(width=width, color='lightgray'), hoverinfo='text', text=([text]), mode='lines')

# lista edge_trace para armazendar todas as arestas do gráfico
edge_trace2 = []
for edge2 in marvel_net2.edges():
    if marvel_net2.edges()[edge2]['weight'] > 0: # evita arestas com peso 0 no gráfico
        char_1 = edge2[0] #nós conectados
        char_2 = edge2[1] #nós conectados
        x0, y0 = layout_marvel2[char_1] # variáveis obtidas a partir das cordenadas dos nós conectados
        x1, y1 = layout_marvel2[char_2]
        trace2  = make_edge2([x0, x1, None], [y0, y1, None], None, width=5*(marvel_net2.edges()[edge2]['weight']/herois_df2['CONT'].max())) # largura calculado normalizando o peso pelo valor máximo da coluna de contagem
        edge_trace2.append(trace2) # chama-mos a função make_edge passando-lhe as coordenadas dos nós(x0...) , o texto como None

### -  **Between Centrality**

### -  **Closeness Centrality**

### -  **Eigenvector Centrality**

### -  **Page Rank**

In [ ]:
cent_df2 = pd.DataFrame(index=list(marvel_net2.nodes()))

In [ ]:
#Between Centrality
between_2 = nx.betweenness_centrality(marvel_net2, weight='weight')
cent_df2['Betweenness Centrality'] = pd.Series(index=[node for node, betweenness_centrality in between_2.items()],data=[float(betweenness_centrality) for node, betweenness_centrality in between_2.items()])

#Closeness Centrality
closeness_2 = nx.closeness_centrality(marvel_net2,distance="weight")   # closeness function tem um parametro distance
cent_df2['Closeness Centrality'] = pd.Series(index=[node for node, closeness_centrality in closeness_2.items()],data=[float(closeness_centrality) for node, closeness_centrality in closeness_2.items()])

#Eigenvector Centrality
eigen_2 = nx.eigenvector_centrality(marvel_net2, weight='weight')
cent_df2['Eigenvector Centrality'] = pd.Series(index=[node for node, eigenvector_centrality in eigen_2.items()],data=[float(eigenvector_centrality) for node, eigenvector_centrality in eigen_2.items()])

#PageRank

page_2 = nx.pagerank(marvel_net2, weight='weight')
cent_df2['PageRank'] = pd.Series(index=[node for node, eigen_centrality in page_2.items()],data=[float(page_centrality) for node, page_centrality in page_2.items()])

display(cent_df2)

,Betweenness Centrality,Closeness Centrality,Eigenvector Centrality,PageRank
THING,0.000000,0.058943,0.360577,0.053607
HUMAN TORCH,0.000000,0.052920,0.360804,0.053320
MR. FANTASTIC,0.000000,0.058943,0.352787,0.051511
WOLVERINE,0.000000,0.185897,0.212297,0.044590
INVISIBLE WOMAN,0.000000,0.089506,0.343165,0.049573
BEAST,0.000000,0.136792,0.208755,0.045463
"WATSON-PARKER, MARY",0.637459,0.290000,0.017924,0.021109
DAREDEVIL,0.000000,0.120833,0.060142,0.017204
HAWK,0.000000,0.157609,0.118610,0.033154
CYCLOPS,0.000000,0.157609,0.237367,0.048034


In [ ]:
# node trace2 para armazenar os nodes do gráfico
node_trace2 = go.Scatter(x=[], y=[], text=[], textposition="top center", textfont_size=10, mode='markers+text', hoverinfo='none',
                        marker=dict(color=[], size=[], line_width=[], line_color=[]))

# Para cada node fazer a posição e tamanho e adicionar a node_trace2
for node2 in marvel_net2.nodes():
    x, y = layout_marvel2[node2]
    node_trace2['x'] += tuple([x])
    node_trace2['y'] += tuple([y])
    color = 'gray'
    line_width = 2
    line_color = 'darkgray'
    name_text = node2


# agora para personalizar os top 10 herois com cores

    if node2 in HERO_COLOR2:
        color = HERO_COLOR2[node2]; line_color='black';

    if node2 in [v[0] for v in page_2]:
        name_text = '<b>' + node2 + '</b>'

    node_trace2['marker']['color'] += tuple([color])
    node_trace2['marker']['size'] += tuple([int(400*page_2[node2])]) # node size is proportional to page rank
    node_trace2['marker']['line_color'] += tuple([line_color])
    node_trace2['text'] += tuple([name_text])



# Layout2 costumizável

layout2 = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # background transparente
    plot_bgcolor='rgba(0,0,0,0)', # 2 background transparente
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)

# Criar figura2
fig2 = go.Figure(layout = layout2)

# Adiciona todas as informação na lista de edge_trace2
for trace2 in edge_trace2:
    fig2.add_trace(trace2)

# Adiciona-se node_trace2  ( tem informação dos nós)
fig2.add_trace(node_trace2)
# Remove-se as etiquetas
fig2.update_layout(showlegend = False)
# Remove-se as etiquetas de x e y
fig2.update_xaxes(showticklabels = False)
fig2.update_yaxes(showticklabels = False)


fig2.update_layout(title=" <b>Network dos top 30 Heróis (depois do SNAP)<b>") # Adiciona-se título e os <b> servem para por a negrito
fig2.show()

# **4-** Comparações

### **A)** Maior número de Conexões

In [ ]:
from IPython.display import display, HTML # Para sair a negrito

In [ ]:
a1 = top_10_heroes.reset_index().rename(columns={'index': 'Hero', 0: 'Connections'})
a2 = top_10_heroes2.reset_index().rename(columns={'index': 'Hero', 0: 'Connections'})

# Ajustar o índice de a1 e a2 para começar em 1
a1.index += 1
a2.index += 1

b1 = "<h2>Top 10 Heróis(antes do SNAP)</h2>\n" + a1.head(10).to_html()
b2= "<h2>Top 10 Heróis(depois do SNAP)</h2>\n" + a2.head(10).to_html()

html = "<table><tr><td style='padding-right:60px'>{}</td><td>{}</td></tr></table>".format(b1,b2)
display(HTML(html))

,Hero,Connections
1,CAPTAIN AMERICA,16499.0
2,SPIDER-MAN,13717.0
3,IRON MAN,11817.0
4,THOR,11427.0
5,THING,10681.0
6,WOLVERINE,10353.0
7,HUMAN TORCH,10237.0
8,SCARLET WITCH,9911.0
9,MR. FANTASTIC,9775.0
10,VISION,9696.0


### **B)** Conexões entre heróis

In [ ]:
sorted_df = herois_df.sort_values(by='CONT', ascending=False)
sorted_df2 = herois_df2.sort_values(by='CONT', ascending=False)

c1 = "<h2>Conexões entre Heróis(antes do SNAP)</h2>\n" + sorted_df.head(10).to_html()
c2= "<h2>Conexões entre Heróis(depois do SNAP)</h2>\n" + sorted_df2.head(10).to_html()

html = "<table><tr><td style='padding-right:60px'>{}</td><td>{}</td></tr></table>".format(c1,c2)
display(HTML(html))

,Hero1,Hero2,CONT
85,THING,HUMAN TORCH,724
135,HUMAN TORCH,MR. FANTASTIC,694
86,THING,MR. FANTASTIC,690
161,MR. FANTASTIC,INVISIBLE WOMAN,682
138,HUMAN TORCH,INVISIBLE WOMAN,675
89,THING,INVISIBLE WOMAN,650
12,SPIDER-MAN,"WATSON-PARKER, MARY",614
18,SPIDER-MAN,"JAMESON, J. JONAH",514
29,CAPTAIN AMERICA,IRON MAN,440
250,SCARLET WITCH,VISION,426


### **C)** Grafos

In [ ]:
fig.update_layout(title=" <b>Network dos Top 30 heróis(antes do SNAP)<b>") # Adiciona-se título e os <b> servem para por a negrito
fig.show()

In [ ]:
fig2.update_layout(title=" <b>Network dos top 30 Heróis (depois do SNAP)<b>") # Adiciona-se título e os <b> servem para por a negrito
fig2.show()

### **D)** -  Between Centrality , Closeness Centrality , Eigenvector Centrality , Page Rank

In [ ]:
d1 = "<h2>Centralidade(antes do snap)</h2>\n" + cent_df.head(20).to_html()
d2= "<h2>Centralidade(depois do SNAP)</h2>\n" + cent_df2.head(20).to_html()

html = "<table><tr><td style='padding-right:60px'>{}</td><td>{}</td></tr></table>".format(d1,d2)
display(HTML(html))

,Betweenness Centrality,Closeness Centrality,Eigenvector Centrality,PageRank
SPIDER-MAN,0.000000,0.030526,0.140174,0.040601
CAPTAIN AMERICA,0.000000,0.041193,0.292802,0.053595
IRON MAN,0.000000,0.060417,0.249492,0.043453
THING,0.000000,0.048986,0.297003,0.046662
THOR,0.000000,0.068396,0.214586,0.036956
HUMAN TORCH,0.000000,0.042899,0.294740,0.046054
MR. FANTASTIC,0.000000,0.042899,0.290314,0.044625
HULK,0.000000,0.078804,0.109661,0.024427
WOLVERINE,0.000000,0.073232,0.142526,0.036160
INVISIBLE WOMAN,0.000000,0.068396,0.278668,0.042289
